In [138]:
# Setting up the environment

import pandas as pd
from dotenv import load_dotenv
import os 

load_dotenv()

df: pd.Series = pd.read_json('db.json',typ='series')

df = df[:5]

from utils.html_tools import extract_text

df = df.apply(lambda x: extract_text(x))
print(df[0])

 Angular Documentation Angular is an application-design framework and development platform for creating efficient and sophisticated single-page apps. These Angular docs help you learn and use the Angular framework and development platform, from your first application to optimizing complex single-page applications for enterprises. Tutorials and guides include downloadable examples to help you start your projects. Assumptions These docs assume that you are already familiar with HTML, CSS, JavaScript, and some of the tools from the latest standards, such as classes and modules. The code samples are written using TypeScript. Most Angular code can be written with just the latest JavaScript, using types for dependency injection, and using decorators for metadata. Feedback We want to hear from you. Report problems or submit suggestions for future docs. Contribute to Angular docs by creating pull requests on the Angular GitHub repository. See Contributing to Angular for information about submi

In [139]:
from utils.open_ai_tools import parse_to_blink
import openai
# OpenAI Integration
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "OOPS, please set env var called 'OPEN_AI_API_KEY'")

In [140]:
# Langchain
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY, max_tokens=-1,model_name="gpt-3.5-turbo-0613")
from langchain import PromptTemplate

template = """
You are a programming teacher. You talk only in json and you dont use newline characters outside of json values.
Generate a json response that teaches something and explains a concept from the provided HTML content (make the lesson universal, IMPORTANT! This is used in a 3rd party app, dont make any references to the original docs), I will kill myself if i see anything else. adhering to this JSON structure: 
(
"title": "<title>",
"description": "<explanation>", # Make this sound human
"code": "<code>"
"difficulty": "<beginner | intermediate | advanced>"
)

HTML content to convert into a blink: {html}
"""

prompt = PromptTemplate(
    input_variables=["html"],
    template=template,
)
from utils.open_ai_tools import truncate_text

blinks = {}
for index, i in enumerate(df.values):
    parsed_text = truncate_text(i, 1800)
    blink_prompt = prompt.format(html=parsed_text)
    blinks[index] = llm.predict(blink_prompt)

/home/fifi/Repositories/lang-blink-engine/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/fifi/Repositories/lang-blink-engine/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:716: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [141]:
blinks

{0: '{\n  "title": "Introduction to Angular",\n  "description": "Angular is a powerful application-design framework and development platform for creating efficient and sophisticated single-page apps. It helps developers in building dynamic and interactive web applications by providing a structured framework and a wide range of features. Angular docs serve as a comprehensive guide to learning and utilizing the Angular framework, starting from basic application development to optimizing complex single-page applications for enterprises. Whether you are a beginner or an experienced developer, these docs offer tutorials, guides, and downloadable examples to kickstart your Angular projects.",\n  "code": "<blink>Angular Documentation</blink> Angular is an application-design framework and development platform for creating efficient and sophisticated single-page apps. These Angular docs help you learn and use the Angular framework and development platform, from your first application to optimiz

In [142]:
# convert blinks to json
import json

def remove_newlines(json_str):
    # Replace newline characters outside of double quotes
    cleansed_str = ""
    inside_quotes = False
    for ch in json_str:
        if ch == '"':
            inside_quotes = not inside_quotes
        if ch != '\n' or inside_quotes:
            cleansed_str += ch

    # Check if the cleansed string is valid JSON
    try:
        json.loads(cleansed_str)
    except json.JSONDecodeError:
        print("Error: The cleansed string is not valid JSON.")
        return json_str  

    return cleansed_str


for index, blink in blinks.items():
    cleansed_blink = remove_newlines(blink)
    blink_json = json.loads(cleansed_blink)
    blinks[index] = blink_json
blinks

{0: {'title': 'Introduction to Angular',
  'description': 'Angular is a powerful application-design framework and development platform for creating efficient and sophisticated single-page apps. It helps developers in building dynamic and interactive web applications by providing a structured framework and a wide range of features. Angular docs serve as a comprehensive guide to learning and utilizing the Angular framework, starting from basic application development to optimizing complex single-page applications for enterprises. Whether you are a beginner or an experienced developer, these docs offer tutorials, guides, and downloadable examples to kickstart your Angular projects.',
  'code': '<blink>Angular Documentation</blink> Angular is an application-design framework and development platform for creating efficient and sophisticated single-page apps. These Angular docs help you learn and use the Angular framework and development platform, from your first application to optimizing com